In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('../train.csv', index_col=0)
df_train.head()

,id,atm_group,address,address_rus,lat,long,target
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277


In [3]:
df_test = pd.read_csv('../test.csv', index_col=0)
df_test.head()

,id,atm_group,address,address_rus,lat,long
0,8096.0,5478.0,"D. 97, UL. KIROVA NOVOKUZNETSK","улица Кирова, 97, Новокузнецк, Кемеровская обл...",53.757824,87.159786
1,2570.0,3185.5,UL. SERGEJA MAKEEVA MOSKVA,NaN,NaN,NaN
2,2428.0,3185.5,"UL.NAHIMOVA, DOM 33 VILJUCHINSK","улица Нахимова, 33, жилой район Рыбачий, Вилюч...",52.918598,158.514038
3,2799.0,3185.5,MKR 3 D 13B SHAR SHARYPOVO,"3-й микрорайон, Шарыпово, Красноярский край, Р...",55.533496,89.178148
4,250.0,1942.0,29 CHICHERINA STR. CHELYABINSK,"улица Чичерина, 29, Челябинск, Россия, 454128",55.179216,61.297729


In [4]:
df = pd.concat([df_train, df_test], sort=False)
df['id'] = df['id'].astype(int)
df.rename(columns={'long': 'lng'}, inplace=True)
df.head()

,id,atm_group,address,address_rus,lat,lng,target
0,8526,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500
1,8532,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710
2,8533,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540
3,8684,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035
4,37,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277


In [5]:
mask = df[['lat', 'lng']].isna().any(axis=1)
print(mask.sum())
df[mask].head()

420


,id,atm_group,address,address_rus,lat,lng,target
40,4510,496.5,PER. BAZARNY 4 SPASSK-DALNY,NaN,NaN,NaN,0.039191
150,347,1022.0,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,NaN,NaN,NaN,-0.145001
182,1591,1942.0,16A BEREJKOVSK NAB MOSCOW,NaN,NaN,NaN,0.080034
192,440,1942.0,113 MIRA AV. TOGLIATTI,NaN,NaN,NaN,-0.082956
214,556,1942.0,133A MOZHAYSKOE H/W. MOSCOW,NaN,NaN,NaN,0.061517


In [6]:
mask = df['address_rus'].isna()
mask = np.logical_and(mask, df[['lat', 'lng']].notna().all(axis=1))
print(mask.sum())
df[mask]

0


,id,atm_group,address,address_rus,lat,lng,target


In [7]:
mask = df['address_rus'] == ''
mask = np.logical_and(mask, df[['lat', 'lng']].notna().all(axis=1))
print(mask.sum())
df[mask]

0


,id,atm_group,address,address_rus,lat,lng,target


In [8]:
df.shape

(8765, 7)

## Достаем адреса и координаты из Яндекс.Геокодера

In [23]:
import requests

In [24]:
from functools import wraps

import time

def timeout(rps=5):
    t_max = 1.0 / rps

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            t_start = time.time()
            res = func(*args, **kwargs)
            t_delta = time.time() - t_start

            if t_delta < t_max:
                time.sleep(t_max - t_delta)
            return res
        return wrapper
    return decorator

In [25]:
with open('yandex-api.key', mode='r') as f_key:
    YA_API_KEY = next(f_key).strip()

In [26]:
def yandex_request_coord(lat, lon):
    r = requests.get("https://geocode-maps.yandex.ru/1.x/",
                     params={"format": "json",
                             "apikey": YA_API_KEY,
                             "geocode": f"{lat},{lon}",
                             "results": 1})
    return r

def yandex_request_address(address):
    r = requests.get("https://geocode-maps.yandex.ru/1.x/",
                     params={"format": "json",
                             "apikey": YA_API_KEY,
                             "geocode": f"{address}",
                             "results": 1})
    return r

@timeout(rps=5)
def yandex_request(args):
    if len(args) == 2:
        return yandex_request_coord(*args)
    elif len(args) == 1:
        return yandex_request_address(*args)
    else:
        raise TypeError('yandex_request(args) takes 1 positional argument. 1 <= len(args) <= 2')

In [13]:
from tqdm import tqdm

In [ ]:
r_yandex = {}

cases = [0 for _ in range(3)]

for r in tqdm(df.itertuples()):
    if not ( np.isnan(r.lat) or np.isnan(r.lng) ):
        args = (r.lng, r.lat)
        cases[0] += 1
    elif not ( np.isnan(r.address_rus) or r.address_rus == '' ):
        args = (r.address_rus, )
        cases[1] += 1
    else:
        args = (r.address, )
        cases[2] += 1
    
    res = yandex_request(args)
    if res.status_code != 200:
        print("FAILED: ", args)
    r_yandex[r.id] = res
    
cases

In [14]:
atm_fixes = {
    347:  'Набережные Челны, Набережночелнинский просп., 6Б',
    8436: 'Бийск, ул. Матросова, дом 23, корпус 1',
    4787: 'Южно-Сахалинск, Коммунистический проспект, 33',
    1964: 'Йошкар-Ола, ул. Красноармейская, 111',
    272:  'Нижний Тагил, Черноисточинское ш., 49',
    338:  'Казань, Краснококшайская ул., 158',
    5095: 'Ярославль, Большая Октябрьская ул., 118/11',
    440:  'Тольятти, ул. Мира, 113',
    556:  'Московская область, Одинцово, Можайское шоссе, 133А',
    4741: 'Челябинск, ул. Братьев Кашириных, 121',
    6548: 'Самара, Московское шоссе, 4Ас1',
    5279: 'Санкт-Петербург, 4-й Верхний переулок, 19',
    5338: 'Саратовская область, Вольск, Коммунистическая улица, 107',
    4473: 'Камчатский край, Елизовский район, село Паратунка, ул. Молчанова,22',
    2288: 'Уфа, Средняя Трактовая улица, 8',
    5639: 'Амурская область, Благовещенск, Красноармейская улица, 123',
    5846: 'Амурская область, Благовещенск, Красноармейская улица, 123',
    5646: 'Чебоксары, Московский проспект, 40',
    6368: 'Удмуртская Республика, Сарапул, улица Раскольникова, 142В',
    7254: 'Владивосток, улица Корабельная Набережная, 2',
    8495: 'Республика Саха (Якутия), село Майя, Советская улица, 25',
    1829: 'Санкт-Петербург, Московский проспект, 74А',
    1830: 'Санкт-Петербург, Московский проспект, 74А',
    5267: 'Санкт-Петербург, 10-я Красноармейская улица, 22',
    5464: 'Санкт-Петербург, Большая морская улица, 30',
    7304: 'Саратовская область, г. Энгельс, Войсковая часть 06987 (6950-я АБ)',
    5513: 'Ярославль, проспект Машиностроителей, 22',
    5648: 'Ростовская область, Азов, проспект Зои Космодемьянской, 88',
    5647: 'Ростовская область, Азов, проспект Зои Космодемьянской, 88',
    5586: 'Чебоксары, проспект Тракторостроителей, 76',
    5699: 'Ростовская область, Азов, проспект Зои Космодемьянской, 88',
    1500: 'Белгород, улица Щорса, 43А',
    1504: 'Санкт-Петербург, Прибрежная улица, 20',
    5047: 'Москва, Серебряническая набережная, 29',
    5347: 'Чебоксары, улица Богдана Хмельницкого, 109к3',
    
    1680: 'Санкт-Петербург, бульвар Новаторов, 8',
    5284: 'Санкт-Петербург, улица Фаворского, 12',
    7145: 'Санкт-Петербург, улица Бабушкина, 10',
    7037: 'Санкт-Петербург, набережная Обводного канала, 120',
    1469: 'Новосибирск, Гусинобродское шоссе, 20',
    5687: 'Иркутская область, Усолье-Сибирское, проспект Красных Партизан, 24',
    5686: 'Иркутская область, Усолье-Сибирское, проспект Красных Партизан, 24',
    5685: 'Иркутская область, Усолье-Сибирское, проспект Красных Партизан, 24',
    7947: 'Пенза, улица Собинова, 7Е',
    7988: 'Пенза, улица Собинова, 7Е',
    8324: 'Тамбов, Интернациональная улица, 16А',
    8325: 'Тамбов, Интернациональная улица, 16А',
    2748: 'Белгород, улица Белгородского Полка, 34',
    2658: 'Белгород, улица Щорса, 8В',
    2662: 'Белгород, улица Королёва, 9А',
    5534: 'Белгород, проспект Богдана Хмельницкого, 81',
    6363: 'Белгород, проспект Богдана Хмельницкого, 137Т',
    1497: 'Белгород, улица Щорса, 64',
    4487: 'Белгород, улица 50-летия Белгородской области, 11',
    2664: 'Белгород, улица Пугачёва, 5',
    8529: 'Южно-Сахалинск, Сахалинская улица, 69',
    8533: 'Южно-Сахалинск, Коммунистический проспект, 32',
    2360: 'Владивосток, улица Героев-Тихоокеанцев, 5А',
    1711: 'Владивосток, проспект 100-летия Владивостока, 51',
    836:  'Владивосток, Океанский проспект, 98А',
    837:  'Владивосток, Океанский проспект, 98А',
    838:  'Владивосток, Океанский проспект, 98А',
    5877: 'Улан-Удэ, Коммунистическая улица, 47А',
    5878: 'Улан-Удэ, Коммунистическая улица, 47А',
    6075: 'Улан-Удэ, Коммунистическая улица, 47А',
    6076: 'Улан-Удэ, Коммунистическая улица, 47А',
    2289: 'Улан-Удэ, улица Гагарина, 25Б',
    1712: 'Красноярск, проспект имени Газеты Красноярский Рабочий, 199',
    2776: 'Красноярск, улица Железнодорожников, 8',
    5350: 'Красноярск, улица 60 лет Октября, 43',
    3612: 'Красноярск, улица 78-й Добровольческой Бригады, 11',
    6464: 'Нижний Новгород, проспект Кораблестроителей, 22Б',
    3700: 'Нижний Новгород, проспект Кораблестроителей, 22Б',
    8276: 'Иркутск, улица Рабочего Штаба, 78',
    6628: 'Иркутск, улица Рабочего Штаба, 29',
    7108: 'Иркутск, улица Рабочего Штаба, 12',
    6404: 'Сургут, бульвар Свободы, 2',
    1260: 'Сургут, улица Иосифа Каролинского, 13',
    1815: 'Сургут, улица 30 лет Победы, 60',
    2725: 'Барнаул, Павловский тракт, 251В',
    2710: 'Барнаул, переулок Ядринцева, 94',
    7046: 'Тверь, улица Скворцова-Степанова, 7',
    7275: 'Чита, улица Костюшко-Григоровича, 5',
    
    1679: 'Санкт-Петербург, Каменноостровский проспект, 42Б',
    1707: 'Санкт-Петербург, Каменноостровский проспект, 42Б',
    1708: 'Санкт-Петербург, Каменноостровский проспект, 42Б',
    1789: 'Санкт-Петербург, Московский проспект, 193',
    4720: 'Кемеровская область, посёлок городского типа Инской, Липецкая улица, 28/1',
    4722: 'Кемеровская область, посёлок городского типа Инской, Липецкая улица, 28/1',
    403:  'Санкт-Петербург, проспект Обуховской Обороны, 305',
    3535: 'Самара, Красноармейская улица, 93',
    3284: 'Улица Комсомольская, 19а, Первоуральск',
    6912: 'Свердловская область, Екатеринбург, микрорайон Юго-Западный, Белореченская улица, 15',
    4427: 'Томская область, село Первомайское, Ленинская улица, 37',
    410:  'Усолье-Сибирское, Иркутская область, проспект Красных Партизан, 53',
    6155: 'Липецк, улица Меркулова, 14',
    8714: 'Нариманов, Улица Центральная, 13',
    2041: 'Москва, Алтуфьевское шоссе, 70к1',
    7232: 'Ульяновск, улица Станкостроителей, 14',
    5183: 'Курск, Октябрьская улица, 80А',
    4874: 'Красноярск, улица 78-й Добровольческой Бригады, 12',
    7495: 'Тимашевск, Краснодарский край, улица Братьев Степановых, 34',
    1702: 'Санкт-Петербург, Московский проспект, 172',
    3305: 'Хабаровск, улица Морозова Павла Леонтьевича, 58',
    3173: 'Москва, Преображенская площадь, 8',
    2208: 'Омск, Интернациональная улица, 43',
    8402: 'Санкт-Петербург, Московский проспект, 220',
    1602: 'Санкт-Петербург, Владимирский проспект, 21',
    6271: 'Санкт-Петербург, Московский просп., 220',
    3261: 'Москва, Старопетровский проезд, 11к1',
    1170: 'Москва, Севастопольский проспект, 11Е',
    5354: 'Красноярск, улица 78-й Добровольческой Бригады, 15',
    
    5530: 'Дзержинск'
}

In [15]:
df_address_fixed = pd.DataFrame(df.loc[df['id'].isin(atm_fixes), ['id', 'address']])
df_address_fixed['address_fix'] = df_address_fixed['id'].map(lambda k: atm_fixes[k])
df_address_fixed.drop(columns=['id'], inplace=True)
df_address_fixed.drop_duplicates()
df_address_fixed.head()

,address,address_fix
2,"KOMMUN. PR., 32 YUZHNO SAKHAL","Южно-Сахалинск, Коммунистический проспект, 32"
150,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,"Набережные Челны, Набережночелнинский просп., 6Б"
192,113 MIRA AV. TOGLIATTI,"Тольятти, ул. Мира, 113"
214,133A MOZHAYSKOE H/W. MOSCOW,"Московская область, Одинцово, Можайское шоссе,..."
255,8 NOVATOROV STR. S.-PETERBURG,"Санкт-Петербург, бульвар Новаторов, 8"


In [16]:
atm_fixes = {r.id: r.address_fix
             for r in pd.merge(df_address_fixed, df[['address', 'id']], on='address').itertuples()}

In [18]:
for id_, address in atm_fixes.items():
    args = (address, )
    
    res = yandex_request(args)
    if res.status_code != 200:
        print("FAILED: ", args)
    r_yandex[id_] = res

In [19]:
import pickle

with open('yandex.dump', mode='wb') as f_dump:
    pickle.dump(r_yandex, f_dump)

In [20]:
import pickle

with open('yandex.dump', mode='rb') as f_dump:
    r_yandex = pickle.load(f_dump)

In [21]:
df_atms = []

for id_, r in r_yandex.items():
    atm_info = {'id': id_}
    
    r = r_yandex[id_].json()['response']['GeoObjectCollection']['featureMember']
    if len(r):
        r = r[0]['GeoObject']

        try:
            lng, lat = list(map(float, r['Point']['pos'].split()))
        except:
            lng, lat = None, None
        finally:
            atm_info['lat'], atm_info['lng'] = lat, lng
                    
        try:
            address = r['metaDataProperty']['GeocoderMetaData']['Address']['formatted']
        except:
            address = ''
        finally:
            atm_info['address'] = address
            
        # """
        try:
            components = r['metaDataProperty']['GeocoderMetaData']['Address']['Components']
            components = {c['kind'] : c['name'] for c in components}
        except:
            components = {}
        finally:
            atm_info.update(components)
        # """
        
    df_atms.append(atm_info)
    
import re    

def find_house_number(house):
    if not len(house):
        return -1
    
    r = re.search('\d+', house)
    if r:
        return int(r.group(0))
    else:
        return -1
    
df_atms = pd.DataFrame(df_atms)
df_atms['house'] = df_atms['house'].fillna('').map(find_house_number)
df_atms.drop(columns=['other', 'railway', 'route', 'area', 'street',
                      'vegetation', 'district', 'hydro', 'airport'], inplace=True)
df_atms.head()

,address,country,house,id,lat,lng,locality,province
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",Россия,34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",Россия,259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",Россия,32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область
3,"Москва, Ленинградский проспект, 76А",Россия,76,8684,55.805837,37.515155,Москва,Москва
4,"Красноярский край, Норильск, Гвардейская площа...",Россия,2,37,69.343541,88.211228,Норильск,Красноярский край


In [22]:
df_atms.isna().any(axis=0)

address     False
country     False
house       False
id          False
lat         False
lng         False
locality     True
province    False
dtype: bool

In [23]:
mask = df_atms[['locality', 'province']].isna().any(axis=1)
df_atms[mask].shape

(95, 8)

In [24]:
mask = df_atms[['lat', 'lng']].isna().any(axis=1)
df_atms[mask]

,address,country,house,id,lat,lng,locality,province


In [25]:
mask = df_atms['country'] != 'Россия'
df_atms[mask]

,address,country,house,id,lat,lng,locality,province
6644,"Онтарио, Юнкнаун-Лейк",Канада,-1,5530,49.783884,-80.652121,NaN,Онтарио


In [26]:
df_atm_info = pd.merge(df_atms, df[['id', 'atm_group', 'target']], on='id')
df_atm_info.head(10)

,address,country,house,id,lat,lng,locality,province,atm_group,target
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",Россия,34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область,32.0,0.011500
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",Россия,259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область,32.0,0.029710
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",Россия,32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область,32.0,0.009540
3,"Москва, Ленинградский проспект, 76А",Россия,76,8684,55.805837,37.515155,Москва,Москва,32.0,-0.094035
4,"Красноярский край, Норильск, Гвардейская площа...",Россия,2,37,69.343541,88.211228,Норильск,Красноярский край,32.0,0.079277
5,"Приморский край, Владивосток, Русская улица, 16",Россия,16,313,43.165860,131.908212,Владивосток,Приморский край,32.0,-0.145001
6,"Москва, улица Шухова, 14",Россия,14,335,55.716859,37.613063,Москва,Москва,32.0,-0.024682
7,"Приморский край, Владивосток, улица Адмирала Ф...",Россия,25,358,43.117174,131.885503,Владивосток,Приморский край,32.0,-0.133426
8,"Московская область, городской округ Истра, дер...",Россия,47,2895,55.799720,37.116393,деревня Веледниково,Московская область,32.0,0.141469
9,"Пенза, Бекешская улица, 39",Россия,39,1975,53.203220,44.978826,Пенза,Пензенская область,496.5,0.017259


In [27]:
mask = np.logical_or(df_atm_info["country"] == "Россия", df_atm_info["target"].isna())
df_atm_info = df_atm_info[mask]
df_atm_info.shape

(8765, 10)

In [28]:
df_atm_info.to_csv('dataset_fixed.csv', sep=',', index=False)

## Достаем данные из OpenStreetMap (Nominatim)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_atm_info = pd.read_csv('dataset_fixed.csv', sep=',')
df_atm_info.head()

,address,country,house,id,lat,lng,locality,province,atm_group,target
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",Россия,34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область,32.0,0.011500
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",Россия,259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область,32.0,0.029710
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",Россия,32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область,32.0,0.009540
3,"Москва, Ленинградский проспект, 76А",Россия,76,8684,55.805837,37.515155,Москва,Москва,32.0,-0.094035
4,"Красноярский край, Норильск, Гвардейская площа...",Россия,2,37,69.343541,88.211228,Норильск,Красноярский край,32.0,0.079277


In [3]:
def osm_fix_address(s):
    i_chosen = -1

    for i, (cp, cc, cn) in enumerate(zip(s[:-2], s[1:-1], s[2:])):
        if cp.isnumeric() and cc.isalpha() and cn.isnumeric():
            i_chosen = i
    
    if i_chosen >= 0:
        s = s[:i+1] + ' ' + s[i+1:]
        
    return s

In [4]:
df_atm_info['address'] = df_atm_info['address'].map(osm_fix_address)

In [5]:
import requests

In [6]:
with open("proxies.tsv", mode='r') as f_proxies:
    proxies = []
    for s in f_proxies:
        ip, t = s.lower().strip().split('\t')
        proxies.append({t : ip})

In [ ]:
proxies

In [8]:
from functools import wraps

import time

def timeout(rps=5):
    t_max = 1.0 / rps

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            t_start = time.time()
            res = func(*args, **kwargs)
            t_delta = time.time() - t_start

            if t_delta < t_max:
                time.sleep(t_max - t_delta)
            return res
        return wrapper
    return decorator

In [9]:
@timeout(rps=1)
def osm_request_coord(sess, lat, lon):
    return sess.get("https://nominatim.openstreetmap.org/reverse",
                    params={"format": "jsonv2", "lat": lat, "lon": lon})

@timeout(rps=1)
def osm_request_address(sess, address):
    return sess.get("https://nominatim.openstreetmap.org/",
                    params={"format": "json", "q": address, "limit": 1, "namedetails": 1})

from queue import Queue

q_in, q_out = Queue(), Queue()

from threading import Lock

n_threads = 0
n_threads_lock = Lock()

def osm_session(proxy, i_thread):
    global n_threads
    
    sess = requests.Session()
    sess.headers.update({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})
    sess.proxies.update(proxy)
    
    with n_threads_lock:
        n_threads += 1
    
    n_processed = 0
    
    while True:
        item = q_in.get()
        if item is None:
            break
            
        id_, address, coords = item
        
        try:
            n_tries = 3
            
            for i in range(n_tries):
                try:
                    r = osm_request_address(sess, address)                    
                    if not len(r.json()):
                        time.sleep(0.5)
                        r = osm_request_coord(sess, *coords)
                except Exception as e:
                    if i + 1 == n_tries:
                        raise e
                else:
                    break
        except Exception as e:
            print(f"Thread {i_thread} has failed: {e}.")
            q_in.put(item)
            q_in.task_done()
            break
            
        if r.status_code != 200:
            print(f"Thread {i_thread} has failed: {r.status_code}.")
            q_in.put(item)
            q_in.task_done()
            break
        
        n_processed += 1
        q_out.put((id_, r))
        q_in.task_done()
        
        if n_processed % 100 == 0:
            print(f"Thread {i_thread} has processed {n_processed} queries. qsize = {q_in.qsize()}")
            time.sleep(1)
            
    with n_threads_lock:
        n_threads -= 1
        print(f"Thread {i_thread} has finished. {n_threads} threads has left. qsize = {q_in.qsize()}")

In [10]:
from tqdm import tqdm

In [ ]:
# проверка прокси серверов

for proxy in tqdm(proxies):
    try:
        r = requests.get('https://ya.ru/', proxies=proxy)
        if r.status_code != 200:
            print(proxy, r.status_code)
    except:
        print(proxy)

In [11]:
for row in df_atm_info.itertuples():
    row = (row.id, row.address, (row.lat, row.lng))
    q_in.put(row)

In [ ]:
%%time

from threading import Thread

pool = [Thread(target=osm_session, args=(proxy, i + 1)) for i, proxy in enumerate(proxies)]

for t in pool:
    t.start()
    
q_in.join()

for t in range(n_threads):
    q_in.put(None)
    
for t in pool:
    t.join()

In [13]:
q_in.qsize(), q_out.qsize()

(0, 8765)

In [14]:
r_osm = []
while not q_out.empty():
    r_osm.append(q_out.get())

In [15]:
all(c[1].status_code == 200 for c in r_osm)

True

In [16]:
import pickle

with open('osm-2.dump', mode='wb') as f_dump:
    pickle.dump(r_osm, f_dump)

In [ ]:
import pickle

with open('osm-2.dump', mode='rb') as f_dump:
    r_osm = pickle.load(f_dump)

In [18]:
df_atm_osm = []

for id_, k in r_osm:
    k = k.json()
    
    if isinstance(k, dict):
        atm_info = {
            'id': id_,
            'importance': float(k['importance']),
            'type': k['category'] + ':' + k['type'],
        }
    
        if k['name'] is not None:
            atm_info['name-osm'] = k['name']

        al, ar, bl, br = tuple(map(float, k['boundingbox']))
        atm_info['square'] = np.abs((al - ar) * (bl - br)) * 111000 ** 2
    elif isinstance(k, list) and len(k) > 0:
        k = k[0]
        
        atm_info = {
            'id': id_,
            'importance': k['importance'],
            'type': k['class'] + ':' + k['type'],
        }
    
        if 'namedetails' in k and 'name' in k['namedetails']:
            atm_info['name-osm'] = k['namedetails']['name']

        al, ar, bl, br = tuple(map(float, k['boundingbox']))
        atm_info['square'] = np.abs((al - ar) * (bl - br)) * 111000 ** 2
    else:
        atm_info = {'id': id_}
        
    df_atm_osm.append(atm_info)
    
df_atm_osm = pd.DataFrame(df_atm_osm)
df_atm_osm[['name-osm', 'type']] = df_atm_osm[['name-osm', 'type']].fillna('')
df_atm_osm[['importance', 'square']] = df_atm_osm[['importance', 'square']].fillna(0)

mask = df_atm_osm['type'].map(lambda s: s.startswith('highway'))
df_atm_osm.loc[mask, 'square'] = 0.0

df_atm_osm['square']
df_atm_osm.head(10)

,id,importance,name-osm,square,type
0,8684,0.421,Метромаркет,7267.718164,shop:mall
1,335,0.421,DeVisu «У Шуховской башни»,123.210000,shop:travel_agency
2,358,0.721,,1982.328401,building:yes
3,1975,0.421,Россельхозбанк,123.210000,amenity:bank
4,1977,0.510,Большая Московская улица,0.000000,highway:primary
5,2006,0.621,ТЦ Цветочный,2588.766665,shop:mall
6,2011,0.811,Россельхозбанк,123.210000,amenity:bank
7,2017,0.520,улица Доржиева,0.000000,highway:secondary
8,2022,0.711,,8603.790031,building:yes
9,2033,0.700,проспект Победы,0.000000,highway:primary


In [19]:
df_atm_osm['name-osm'] = df_atm_osm.apply(
    lambda r: '' if r['type'].startswith('highway') else r['name-osm'], axis=1)

In [20]:
(df_atm_osm['name-osm'] == '').sum(), (df_atm_osm['type'] == '').sum()

(3798, 0)

In [21]:
df_atm_osm['type'].unique().shape

(211,)

In [22]:
df_atm = pd.merge(df_atm_info, df_atm_osm, on='id')
df_atm.head()

,address,country,house,id,lat,lng,locality,province,atm_group,target,importance,name-osm,square,type
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",Россия,34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область,32.0,0.011500,0.000,Островной,1823.367048,shop:supermarket
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",Россия,259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область,32.0,0.029710,0.711,,1492.422770,building:yes
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",Россия,32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область,32.0,0.009540,0.711,Дом правительства Сахалинской области,14004.437943,office:government
3,"Москва, Ленинградский проспект, 76А",Россия,76,8684,55.805837,37.515155,Москва,Москва,32.0,-0.094035,0.421,Метромаркет,7267.718164,shop:mall
4,"Красноярский край, Норильск, Гвардейская площа...",Россия,2,37,69.343541,88.211228,Норильск,Красноярский край,32.0,0.079277,0.621,Управление комбината,24155.432744,office:company


In [23]:
df_atm.drop(columns=['country'], inplace=True)

In [24]:
df_atm.to_csv('dataset_fixed-2.csv', sep=',', index=False)

## Достаем данные о население в субъектах России

In [1]:
import requests
from bs4 import BeautifulSoup

import re
import pandas as pd
import numpy as np

In [2]:
df_atm = pd.read_csv('dataset_fixed-2.csv', sep=',')
df_atm.head()

,address,house,id,lat,lng,locality,province,atm_group,target,importance,name-osm,square,type
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область,32.0,0.011500,0.000,Островной,1823.367048,shop:supermarket
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область,32.0,0.029710,0.711,NaN,1492.422770,building:yes
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область,32.0,0.009540,0.711,Дом правительства Сахалинской области,14004.437943,office:government
3,"Москва, Ленинградский проспект, 76А",76,8684,55.805837,37.515155,Москва,Москва,32.0,-0.094035,0.421,Метромаркет,7267.718164,shop:mall
4,"Красноярский край, Норильск, Гвардейская площа...",2,37,69.343541,88.211228,Норильск,Красноярский край,32.0,0.079277,0.621,Управление комбината,24155.432744,office:company


In [3]:
r = requests.get('https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8')

soup = BeautifulSoup(r.text, 'html.parser')
df_population = pd.read_html(str(soup.find('table', class_='standard')), header=0)[0]
df_population = df_population[df_population.columns[[1,9]]]
df_population.columns = ['subject', 'density']

def process_subject(sub):
    sub = re.sub('\(.*?\)', '', sub)
    sub = re.sub('без (:?.*?)', '', sub)
    sub = sub.strip()
    
    if sub == 'Республика Саха':
        sub = 'Республика Саха (Якутия)'
    elif sub == 'Республика СевернаяОсетия — Алания':
        sub = 'Республика Северная Осетия — Алания'
    elif sub == 'Ханты-Мансийскийавтономный округ — Югра':
        sub = 'Ханты-Мансийский автономный округ'
    
    return sub

df_population['subject'] = df_population['subject'].map(process_subject)

df_population.head(10)

,subject,density
0,Москва,488343
1,Московская область,16927
2,Краснодарский край,7423
3,Санкт-Петербург,381464
4,Свердловская область,2226
5,Ростовская область,4180
6,Республика Башкортостан,2843
7,Республика Татарстан,5740
8,Тюменская область,252
9,Челябинская область,3946


In [4]:
province_set = set(df_atm['province']).difference(set(df_population['subject']))
province_set

set()

In [5]:
df_atm[df_atm['province'].isin(province_set)]

,address,house,id,lat,lng,locality,province,atm_group,target,importance,name-osm,square,type


In [6]:
df_atm = pd.merge(df_atm, df_population, left_on='province', right_on='subject')
df_atm.rename(columns={'density': 'density_province'}, inplace=True)
df_atm['mul_square_density_province'] = np.log(1.0 + df_atm['square'] * df_atm['density_province'])
df_atm['square'] = np.log(1.0 + df_atm['square'])
df_atm.drop('subject', axis=1, inplace=True)
df_atm.head()

,address,house,id,lat,lng,locality,province,atm_group,target,importance,name-osm,square,type,density_province,mul_square_density_province
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область,32.0,0.011500,0.000,Островной,7.508988,shop:supermarket,563,13.841721
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область,32.0,0.029710,0.711,NaN,7.308826,building:yes,563,13.641437
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область,32.0,0.009540,0.711,Дом правительства Сахалинской области,9.547201,office:government,563,15.880409
3,"Сахалинская область, Южно-Сахалинск, Милицейск...",8,470,46.966405,142.731815,Южно-Сахалинск,Сахалинская область,1942.0,0.041914,0.711,NaN,7.477486,building:yes,563,13.810201
4,"Сахалинская область, Южно-Сахалинск, 2-я Центр...",1,1183,46.900367,142.742227,Южно-Сахалинск,Сахалинская область,1942.0,0.065990,0.900,NaN,0.000000,highway:residential,563,0.000000


In [7]:
df_atm['target'].isna().sum()

2504

In [8]:
df_atm.describe()

,house,id,lat,lng,atm_group,target,importance,square,density_province,mul_square_density_province
count,8765.000000,8765.000000,8765.000000,8765.000000,8765.000000,6261.000000,8765.000000,8765.000000,8765.000000,8765.000000
mean,44.785853,4404.586195,54.162875,63.155054,4407.008842,0.000715,0.489027,5.751008,67756.236167,12.605835
std,68.329928,2542.532345,4.313298,32.668376,2427.934099,0.085852,0.219369,3.647240,160211.342539,6.475609
min,-1.000000,1.000000,42.057080,19.941530,32.000000,-0.145001,0.000000,0.000000,7.000000,0.000000
25%,7.000000,2207.000000,52.262764,37.768561,1942.000000,-0.061176,0.421000,4.821974,1162.000000,11.585835
50%,23.000000,4403.000000,55.095812,48.480037,5478.000000,-0.015573,0.511000,4.821974,2843.000000,13.469106
75%,57.000000,6603.000000,56.116673,84.938540,5478.000000,0.040149,0.621000,8.132624,5962.000000,16.719005
max,1805.000000,8811.000000,69.495093,173.706490,8083.000000,0.218608,1.471741,26.912757,488343.000000,35.179800


### Расстояния до станций метро

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [10]:
with open('open-data-moscow.key', mode='r') as f_key:
    ODM_API_KEY = next(f_key).strip()

In [11]:
r = requests.get('https://apidata.mos.ru/v1/datasets/624/rows', params={'api_key': ODM_API_KEY})

In [12]:
df_metro = []

for i in r.json():
    metro_info = {
        'id': i['global_id'],
        'name': i['Cells']['NameOfStation']
    }
    metro_info['lng'], metro_info['lat'] = i['Cells']['geoData']['coordinates']
    df_metro.append(metro_info)
    
df_metro = pd.DataFrame(df_metro)
df_metro.drop_duplicates('id', inplace=True)
df_metro.drop(columns=['id'], inplace=True)

print(df_metro.shape)
df_metro.head()

(1031, 3)


,lat,lng,name
0,55.756767,37.630985,Китай-город
1,55.757315,37.631677,Китай-город
3,55.756882,37.631270,Китай-город
4,55.757236,37.631951,Китай-город
5,55.753198,37.633105,Китай-город


In [13]:
r = requests.get('https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%81%D1%82%D0%B0%D0%BD%D1%86%D0%B8%D0%B9_%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%D1%81%D0%BA%D0%BE%D0%B3%D0%BE_%D0%BC%D0%B5%D1%82%D1%80%D0%BE%D0%BF%D0%BE%D0%BB%D0%B8%D1%82%D0%B5%D0%BD%D0%B0')
soup = BeautifulSoup(r.text, 'html.parser')

In [14]:
table = soup.find_all('table')[3].find('tbody')

df_spb = []
for row in table.find_all('tr'):
    try:
        metro_info = {
            'name': row.find_all('td')[1].text.strip(),
            'lat':  row.find('a', class_='mw-kartographer-maplink')['data-lat'],
            'lng':  row.find('a', class_='mw-kartographer-maplink')['data-lon'],
        }
        df_spb.append(metro_info)
    except:
        pass
    
df_spb = pd.DataFrame(df_spb)
df_spb.head()

,lat,lng,name
0,60.0503972,30.4424083,Девяткино до 1 июля 1992 «Комсомольская»
1,60.0350028,30.418075,Гражданский проспект
2,60.0127194,30.396025,Академическая
3,60.008875,30.3708667,Политехническая
4,59.9999194,30.3666833,Площадь Мужества


In [15]:
df_metro = pd.concat([df_metro, df_spb])
df_metro.head()

,lat,lng,name
0,55.7568,37.631,Китай-город
1,55.7573,37.6317,Китай-город
3,55.7569,37.6313,Китай-город
4,55.7572,37.632,Китай-город
5,55.7532,37.6331,Китай-город


In [16]:
ya_queries = []

In [17]:
r = requests.get('http://metronn.ru/')
soup = BeautifulSoup(r.text, 'html.parser')
df_nn = pd.read_html(str(soup.find('div', class_='stationsshed')))[0]

for st in df_nn['Станция']:
    ya_queries.append(f'Нижний Новгород, станция метро {st}')

In [18]:
r = requests.get('http://www.nsk-metro.ru/index.php?event=menu&pname=Map')
soup = BeautifulSoup(r.text, 'html.parser')
df_nsk = soup.find('map').find_all('area', shape='circle')
df_nsk = [s['alt'] for s in df_nsk]

for st in df_nsk:
    ya_queries.append(f'Новсибирск, станция метро {st}')

In [19]:
r = requests.get('http://metro-ektb.ru/informaciya_o_stanciyah_ekaterinburgskogo_metropolitena/')
soup = BeautifulSoup(r.text, 'html.parser')
st = soup.find('div', class_='uss_photoalbums_albums').find_all('div', class_='uss_albums_name')

df_ekb = [s.find('a', href=True).text for s in st]

for st in df_ekb:
    ya_queries.append(f'Екатеринбург, станция метро {st}')

In [20]:
r = requests.get('http://metrosamara.ru/passengers/scheme/')
soup = BeautifulSoup(r.text, 'html.parser')

st = soup.find('div', class_='sidebar').find('li', class_='active').find_all('li')
df_sam = [s.text for s in st]

for st in df_ekb:
    ya_queries.append(f'Самара, станция метро {st}')

In [21]:
r = requests.get('http://k-metro.ru/stations/')
soup = BeautifulSoup(r.text, 'html.parser')

df_kzn = pd.read_html(str(soup.find('table')), header=0)[0]

for st in df_kzn['Станция']:
    ya_queries.append(f'Казань, станция метро {st}')

In [22]:
ya_queries

['Нижний Новгород, станция метро Стрелка',
 'Нижний Новгород, станция метро Горьковская',
 'Нижний Новгород, станция метро Буревестник',
 'Нижний Новгород, станция метро Бурнаковская',
 'Нижний Новгород, станция метро Канавинская',
 'Нижний Новгород, станция метро Московская',
 'Нижний Новгород, станция метро Чкаловская',
 'Нижний Новгород, станция метро Ленинская',
 'Нижний Новгород, станция метро Заречная',
 'Нижний Новгород, станция метро Двигатель Революции',
 'Нижний Новгород, станция метро Пролетарская',
 'Нижний Новгород, станция метро Автозаводская',
 'Нижний Новгород, станция метро Комсомольская',
 'Нижний Новгород, станция метро Кировская',
 'Нижний Новгород, станция метро Парк Культуры',
 'Новсибирск, станция метро Маркса',
 'Новсибирск, станция метро Студенческая',
 'Новсибирск, станция метро Речной Вокзал',
 'Новсибирск, станция метро Октябрьская',
 'Новсибирск, станция метро Пл.Ленина',
 'Новсибирск, станция метро Золотая Нива',
 'Новсибирск, станция метро Березовая Роща'

In [27]:
df_ya = []

for q in ya_queries:
    r = yandex_request((q,))
    
    if r.status_code != 200:
        print("FAIL", r.url)
        break
        
    metro_info = {'name': re.search('метро (.*?)$', q).group(1)}
    
    r = r.json()['response']['GeoObjectCollection']['featureMember']
    if len(r):
        r = r[0]['GeoObject']

        try:
            lng, lat = list(map(float, r['Point']['pos'].split()))
        except:
            lng, lat = None, None
        finally:
            metro_info['lat'], metro_info['lng'] = lat, lng
            
    df_ya.append(metro_info)
    
df_ya = pd.DataFrame(df_ya)
df_ya.head()

,lat,lng,name
0,56.335286,43.949545,Стрелка
1,56.313958,43.994317,Горьковская
2,56.333290,43.894784,Буревестник
3,56.325779,43.911978,Бурнаковская
4,56.320737,43.926683,Канавинская


In [28]:
df_metro = pd.concat([df_metro, df_ya])

In [29]:
df_metro[['lat', 'lng']] = df_metro[['lat', 'lng']].astype(float)

In [30]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier(metric='haversine')
knc.fit(np.deg2rad(df_metro[['lat', 'lng']]).values, np.ones(df_metro.shape[0]))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='haversine',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [31]:
distances, indexes = knc.kneighbors(X=np.deg2rad(df_atm[['lat', 'lng']]).values, n_neighbors=1)

In [32]:
R = 6373.0 # радиус земли в километрах

distances *= R
distances

array([[ 4145.2630798 ],
       [ 4146.42969007],
       [ 4144.07192367],
       ..., 
       [ 1176.6413421 ],
       [ 1199.44788607],
       [ 1198.57055276]])

In [33]:
df_atm['metro'] = distances
df_atm.head()

,address,house,id,lat,lng,locality,province,atm_group,target,importance,name-osm,square,type,density_province,mul_square_density_province,metro
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область,32.0,0.011500,0.000,Островной,7.508988,shop:supermarket,563,13.841721,4145.263080
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область,32.0,0.029710,0.711,NaN,7.308826,building:yes,563,13.641437,4146.429690
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область,32.0,0.009540,0.711,Дом правительства Сахалинской области,9.547201,office:government,563,15.880409,4144.071924
3,"Сахалинская область, Южно-Сахалинск, Милицейск...",8,470,46.966405,142.731815,Южно-Сахалинск,Сахалинская область,1942.0,0.041914,0.711,NaN,7.477486,building:yes,563,13.810201,4143.232356
4,"Сахалинская область, Южно-Сахалинск, 2-я Центр...",1,1183,46.900367,142.742227,Южно-Сахалинск,Сахалинская область,1942.0,0.065990,0.900,NaN,0.000000,highway:residential,563,0.000000,4148.109397


### Расстояния до центров субъектов и городов

In [34]:
df_atm[["locality", "province"]] = df_atm[["locality", "province"]].fillna('')

In [35]:
from collections import Counter

len(set(zip(df_atm["locality"], df_atm["province"]))), len(df_atm["province"].unique())

(1011, 80)

In [36]:
from tqdm import tqdm, tqdm_notebook

In [37]:
ya_result = {}

for p in tqdm(set(zip(df_atm["locality"], df_atm["province"]))):
    if p[0] == p[1]:
        args = (p[0], )
    else:
        args = (f"{p[1]}, {p[0]}", )
        
    r = yandex_request(args)
    if r.status_code != 200:
        print("FAIL", args, r.status_code)
        break
        
    ya_result[p] = r

100%|██████████| 1011/1011 [03:47<00:00,  4.44it/s]


In [38]:
df_centers_pair = []

for k, v in ya_result.items():
    center_info = {
        'locality': k[0],
        'province': k[1],
    }
    
    r = v.json()['response']['GeoObjectCollection']['featureMember']
    if len(r):
        r = r[0]['GeoObject']

        try:
            lng, lat = list(map(float, r['Point']['pos'].split()))
        except:
            lng, lat = None, None
        finally:
            center_info['lat'], center_info['lng'] = lat, lng
            
    df_centers_pair.append(center_info)
    
df_centers_pair = pd.DataFrame(df_centers_pair)
df_centers_pair.head()

,lat,lng,locality,province
0,54.955908,83.177518,село Барышево,Новосибирская область
1,44.597641,132.817559,Спасск-Дальний,Приморский край
2,55.678859,37.263986,Одинцово,Московская область
3,51.159252,37.886214,садовые участки Новиково,Белгородская область
4,54.782635,32.045251,Смоленск,Смоленская область


In [39]:
df_centers_pair_index = df_centers_pair.set_index(['locality', 'province'])
df_centers_pair_index[['lat', 'lng']] = np.deg2rad(df_centers_pair_index[['lat', 'lng']])
df_centers_pair_index.head()

,,lat,lng
locality,province,,
село Барышево,Новосибирская область,0.959162,1.451722
Спасск-Дальний,Приморский край,0.778376,2.318104
Одинцово,Московская область,0.971779,0.650379
садовые участки Новиково,Белгородская область,0.892897,0.661239
Смоленск,Смоленская область,0.956137,0.559295


In [40]:
from sklearn.neighbors import DistanceMetric

In [41]:
R = 6373.0 # радиус Земли в километрах

metric = DistanceMetric.get_metric('haversine')

df_atm['distance_locality'] = 0.0

for p in set(zip(df_atm["locality"], df_atm["province"])):
    mask = np.logical_and(df_atm['locality'] == p[0], df_atm['province'] == p[1])
    coords = np.deg2rad(df_atm[['lat', 'lng']][mask])
    dist = metric.pairwise(coords.values,
                           df_centers_pair_index.loc[p].values.reshape(-1, 2)) * R
    df_atm.loc[mask, 'distance_locality'] = dist[:,0]

In [42]:
df_centers_pair.rename(columns={'lat': 'lat_locality', 'lng': 'lng_locality'}, inplace=True)

In [43]:
df_atm = pd.merge(df_atm, df_centers_pair, on=['locality', 'province'])
df_atm.head()

,address,house,id,lat,lng,locality,province,atm_group,target,importance,name-osm,square,type,density_province,mul_square_density_province,metro,distance_locality,lat_locality,lng_locality
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область,32.0,0.011500,0.000,Островной,7.508988,shop:supermarket,563,13.841721,4145.263080,2.020059,46.959155,142.738023
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область,32.0,0.029710,0.711,NaN,7.308826,building:yes,563,13.641437,4146.429690,2.690682,46.959155,142.738023
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область,32.0,0.009540,0.711,Дом правительства Сахалинской области,9.547201,office:government,563,15.880409,4144.071924,0.144748,46.959155,142.738023
3,"Сахалинская область, Южно-Сахалинск, Милицейск...",8,470,46.966405,142.731815,Южно-Сахалинск,Сахалинская область,1942.0,0.041914,0.711,NaN,7.477486,building:yes,563,13.810201,4143.232356,0.934019,46.959155,142.738023
4,"Сахалинская область, Южно-Сахалинск, 2-я Центр...",1,1183,46.900367,142.742227,Южно-Сахалинск,Сахалинская область,1942.0,0.065990,0.900,NaN,0.000000,highway:residential,563,0.000000,4148.109397,6.546772,46.959155,142.738023


In [44]:
ya_result = {}

for p in tqdm(set(df_atm["province"])):
    args = (p, )
        
    r = yandex_request(args)
    if r.status_code != 200:
        print("FAIL", args, r.status_code)
        break
        
    ya_result[p] = r

100%|██████████| 80/80 [00:17<00:00,  4.63it/s]


In [45]:
df_centers = []

for k, v in ya_result.items():
    center_info = { 'province': k }
    
    r = v.json()['response']['GeoObjectCollection']['featureMember']
    if len(r):
        r = r[0]['GeoObject']

        try:
            lng, lat = list(map(float, r['Point']['pos'].split()))
        except:
            lng, lat = None, None
        finally:
            center_info['lat'], center_info['lng'] = lat, lng
            
    df_centers.append(center_info)
    
df_centers = pd.DataFrame(df_centers)
df_centers.head()

,lat,lng,province
0,61.350179,169.782981,Камчатский край
1,46.851463,47.466189,Астраханская область
2,50.872231,37.303198,Белгородская область
3,58.307715,32.490222,Новгородская область
4,62.575815,154.036835,Магаданская область


In [46]:
df_centers_index = df_centers.set_index(['province'])
df_centers_index[['lat', 'lng']] = np.deg2rad(df_centers_index[['lat', 'lng']])
df_centers_index.head()

,lat,lng
province,,
Камчатский край,1.070763,2.963272
Астраханская область,0.817712,0.828441
Белгородская область,0.887888,0.651064
Новгородская область,1.017662,0.567061
Магаданская область,1.092154,2.688450


In [47]:
R = 6373.0 # радиус Земли в километрах

metric = DistanceMetric.get_metric('haversine')

df_atm['distance_province'] = 0.0

for p in set(df_atm["province"]):
    mask = df_atm['province'] == p
    coords = np.deg2rad(df_atm[['lat', 'lng']][mask])
    dist = metric.pairwise(coords.values,
                           df_centers_index.loc[p].values.reshape(-1, 2)) * R
    df_atm.loc[mask, 'distance_province'] = dist[:,0]

In [48]:
df_centers.rename(columns={'lat': 'lat_province', 'lng': 'lng_province'}, inplace=True)

In [49]:
df_atm = pd.merge(df_atm, df_centers, on=['province'])
df_atm.head()

,address,house,id,lat,lng,locality,province,atm_group,target,importance,...,type,density_province,mul_square_density_province,metro,distance_locality,lat_locality,lng_locality,distance_province,lat_province,lng_province
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область,32.0,0.011500,0.000,...,shop:supermarket,563,13.841721,4145.263080,2.020059,46.959155,142.738023,357.041271,50.150926,142.750797
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область,32.0,0.029710,0.711,...,building:yes,563,13.641437,4146.429690,2.690682,46.959155,142.738023,357.446573,50.150926,142.750797
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область,32.0,0.009540,0.711,...,office:government,563,15.880409,4144.071924,0.144748,46.959155,142.738023,354.897417,50.150926,142.750797
3,"Сахалинская область, Южно-Сахалинск, Милицейск...",8,470,46.966405,142.731815,Южно-Сахалинск,Сахалинская область,1942.0,0.041914,0.711,...,building:yes,563,13.810201,4143.232356,0.934019,46.959155,142.738023,354.216493,50.150926,142.750797
4,"Сахалинская область, Южно-Сахалинск, 2-я Центр...",1,1183,46.900367,142.742227,Южно-Сахалинск,Сахалинская область,1942.0,0.065990,0.900,...,highway:residential,563,0.000000,4148.109397,6.546772,46.959155,142.738023,361.559686,50.150926,142.750797


### Средняя заработная плата в регионе

Проект Росстата за 2018 г.: 
http://www.gks.ru/free_doc/doc_2018/region/soc-pok.rar

In [50]:
df_salary = pd.read_csv('salary.csv', header=None)
df_salary.columns = ['province', 'salary']
df_salary.head()

,province,salary
0,Белгородская область,29066.0
1,Брянская область,24743.0
2,Владимирская область,26975.0
3,Воронежская область,28007.0
4,Ивановская область,23470.0


In [51]:
df_atm = pd.merge(df_atm, df_salary, on='province')
df_atm['salary'] = df_atm['salary'].fillna(np.nanmean(df_atm['salary']))
df_atm.head()

,address,house,id,lat,lng,locality,province,atm_group,target,importance,...,density_province,mul_square_density_province,metro,distance_locality,lat_locality,lng_locality,distance_province,lat_province,lng_province,salary
0,"Сахалинская область, Южно-Сахалинск, улица А.О...",34,8526,46.940995,142.738319,Южно-Сахалинск,Сахалинская область,32.0,0.011500,0.000,...,563,13.841721,4145.263080,2.020059,46.959155,142.738023,357.041271,50.150926,142.750797,68496.0
1,"Сахалинская область, Южно-Сахалинск, Комсомоль...",259,8532,46.937341,142.753339,Южно-Сахалинск,Сахалинская область,32.0,0.029710,0.711,...,563,13.641437,4146.429690,2.690682,46.959155,142.738023,357.446573,50.150926,142.750797,68496.0
2,"Сахалинская область, Южно-Сахалинск, Коммунист...",32,8533,46.960268,142.739011,Южно-Сахалинск,Сахалинская область,32.0,0.009540,0.711,...,563,15.880409,4144.071924,0.144748,46.959155,142.738023,354.897417,50.150926,142.750797,68496.0
3,"Сахалинская область, Южно-Сахалинск, Милицейск...",8,470,46.966405,142.731815,Южно-Сахалинск,Сахалинская область,1942.0,0.041914,0.711,...,563,13.810201,4143.232356,0.934019,46.959155,142.738023,354.216493,50.150926,142.750797,68496.0
4,"Сахалинская область, Южно-Сахалинск, 2-я Центр...",1,1183,46.900367,142.742227,Южно-Сахалинск,Сахалинская область,1942.0,0.065990,0.900,...,563,0.000000,4148.109397,6.546772,46.959155,142.738023,361.559686,50.150926,142.750797,68496.0


In [52]:
df_atm.to_csv('dataset_fixed-3.csv', sep=',', index=False)

In [53]:
df_atm.columns

Index(['address', 'house', 'id', 'lat', 'lng', 'locality', 'province',
       'atm_group', 'target', 'importance', 'name-osm', 'square', 'type',
       'density_province', 'mul_square_density_province', 'metro',
       'distance_locality', 'lat_locality', 'lng_locality',
       'distance_province', 'lat_province', 'lng_province', 'salary'],
      dtype='object')

In [54]:
df_atm['target'].isna().sum()

2504

In [ ]:
raise Exception()

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [ ]:
from collections import Counter
from itertools import chain

In [ ]:
df_atm = pd.read_csv('dataset_fixed-3.csv', sep=',')

https://www.rosbank.ru/files/p2p/P2Pb.xls

In [ ]:
df_p2p = pd.read_csv('P2Pb.csv', sep=',')
df_p2p['atm_id'] = df_p2p['atm_id'].map(lambda s: re.search('\d+', s).group(0))
df_p2p['is24hour'] = (df_p2p['is24hour'] == '24x7').astype(int)
df_p2p['address'] = df_p2p['address'].map(lambda s: re.sub('Район \w+', '', s))
df_p2p.head()

In [ ]:
r_yandex = []

for address in df_p2p['address']:
    args = (address, )
    
    res = yandex_request(args)
    if res.status_code != 200:
        print("FAILED: ", args)
    r_yandex.append(res)